# imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lets_plot import *

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [3]:
from scipy.stats import norm, uniform

In [4]:
from copy import copy

In [5]:
import random

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

In [7]:
def gg_confusion_matrix(y, y_hat):
    conf_mat = confusion_matrix(y, y_hat)[::-1]
    confusion_dat = pd.DataFrame(conf_mat)
    observed = confusion_dat.columns.values
    actual = confusion_dat.index.values
    xx, yy = np.meshgrid(actual, observed)
    xx = xx.reshape(-1)
    yy = yy.reshape(-1)
    zz = conf_mat.reshape(-1)
    dat = {'predicted':xx, 'actual':yy[::-1], 'z':zz}
    p = ggplot(dat, aes('predicted', 'actual', fill='z')) \
        + geom_raster() \
        + geom_text(aes(label='z'), color='white')\
        + theme(legend_position='none', axis_ticks='blank', axis_line='blank')\
        + ggsize(600, 600) + scale_x_discrete() + scale_y_discrete()\
        + ggtitle('Confusion matrix')
    return p

In [8]:
def intersect(a, b):
    return list(set(a) & set(b))

In [9]:
def plot_digit(digit, caption=None):
    digit = digit.reshape(28, 28)
    digit = (digit - np.min(digit))/(np.max(digit) - np.min(digit))
    p = ggplot() + geom_image(image_data=digit) + labs(x='', y='') \
        + theme(axis_line='blank', axis_title='blank', axis_ticks='blank', axis_text='blank')
    if caption:
        p += ggtitle(caption)
    return p

$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\D}{\mathbb{D}}$
$\newcommand{\pr}{\mathbb{P}}$
$\newcommand{\I}{\mathbb{I}}$

$\newcommand{\Er}{\mathcal{E}}$
$\newcommand{\Xset}{\mathcal{X}}$
$\newcommand{\Yset}{\mathcal{Y}}$
$\newcommand{\L}{\mathcal{L}}$
$\newcommand{\l}{\mathcal{l}}$
$\newcommand{\pr}{\mathbb{P}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\D}{\mathbb{D}}$
$\newcommand{\w}{\mathbf{w}}$
$\newcommand{\X}{\mathbf{X}}$
$\newcommand{\y}{\mathbf{y}}$
$\newcommand{\x}{\mathbf{x}}$
$\newcommand{\J}{\mathbf{J}}$
$\newcommand{\I}{\mathbf{I}}$
$\newcommand{\X}{\mathbf{X}}$
$\newcommand{\S}{\mathbf{S}}$
$\newcommand{\e}{\mathbf{e}}$
$\newcommand{\C}{\mathbf{C}}$
$\newcommand{\K}{\mathbf{K}}$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\v}{\mathbf{v}}$
$\newcommand{\phivec}{\boldsymbol{\phi}}$
$\newcommand{\sign}{\mathrm{sign}}$
$\newcommand{\F}{\mathcal{F}}$

$\newcommand{\tg}{\mathrm{tg}}$
$\newcommand{\ctg}{\mathrm{ctg}}$
$\newcommand{\arctg}{\mathrm{arctg}}$

# Logistic regression

### Logistic regression and decision boundary
In this section we are going to consider the logistic regression from the point of view of construction decision boundaries
separating classes.
Suppose in the binary classification task
$\begin{align}
&\mathbb{P}\left(y=1 \;|\; x;\, \theta \right) = a_{\theta}(x)\\
&\mathbb{P}\left(y=0 \;|\; x;\, \theta \right) = 1 - a_{\theta}(x)
\end{align}$
Suppose we are going to find decision boundary that separates two classes:
$$\pr\left(y=0\;|\;x;\theta\right)=\pr\left(y=1\;|\;x;\theta\right)$$
Let's rewrite this statement in the logarithmic form 
$$\log\left(\frac{\pr\left(y=0\;|\;x:\theta\right)}{\pr\left(y=1\;|\;x;\theta\right)}\right)=0$$
Consider the linear equation for the decision boundary:
$$\log\left(\frac{\pr\left(y=0\;|\;x:\theta\right)}{\pr\left(y=1\;|\;x;\theta\right)}\right)=\theta^Tx$$ 
Using the sigmoid function $g(z)$ where $z=\theta^Tx$ and $z^{(i)}=\theta^Tx^{(i)}$ we get that
$$\pr\left(y=1\;|\;x;\theta\right)=g(z)\\
\pr\left(y=0\;|\;x;\theta\right)=1-g(z)$$
Consider the log-likelihood function
$$\ell(\theta)=\sum_{i=1}^m\log p(y^{(i)}\;|\;x^{(i)};\theta)=\sum_{i=1}^m\log\left( g(z^{(i)})^{y^{(i)}}\left(1-g(z^{(i)})\right)^{1-y^{(i)}}\right)=\\
\sum_{i=1}^m\left(y^{(i)}\log g(z^{(i)}) + (1-y^{(i)})\log\left(1-g(z^{(i)})\right)\right)$$

**Gradient of the loss function**
Consider the loss function $J(a,y;\theta)$ as a function of $\theta$: $J(a,y;\theta)=\ell(\theta)$
Let $a_{\theta}(x)=g(z)=g(\theta^Tx)$, where $g(z)$ - sigmoid function.
Set $m=1$ for simplicity and the function takes form 
$$\displaystyle \ell(\theta)=y\log g(\theta^Tx)+(1-y)\log(1-g(\theta^Tx))$$
Take the derivative:
$$
\frac{\partial \ell(\theta)}{\partial \theta_j}=\left(\frac{y}{g(z)}g(z)(1-g(z))-\frac{1-y}{1-g(z)}g(z)(1-g(z))\right)\frac{\partial \theta^Tx}{\partial \theta_j}
=\left(y-g(\theta^Tx)\right)x_j
$$

**Stochastic gradient ascent**
As far as we are working with the likelihood function - our aim is to find it's maximum and therefore we
are going to implement gradient ascent:
$$\boxed{\;\theta_j \leftarrow \theta_j+\eta\left(y^{(i)}-g(\theta^Tx^{(i)})\right)x_j^{(i)}\;}$$

In [10]:
def sigmoid(x,slope=1.0):
    return 1.0/(1.0+np.exp(-slope*x))

In [11]:
def sigmoid_prime(x,slope=1.0):
    return slope*sigmoid(x,slope=slope)*(1.0-sigmoid(x,slope=slope))

In [12]:
def check_gradients(f, grad, z, eps=1e-4, type='central', rtol=1e-5):
    if type == 'forward':
        num_grad = (f(z+eps)-f(z))/(eps)
    elif type=='backward':
        num_grad = (f(z)-f(z-eps))/(eps)
    elif type=='central':
        num_grad = (f(z+eps)-f(z-eps))/(2*eps)
    else:
        raise NotImplementedError
    rel_tol = np.abs(num_grad - grad(z))/(1. + np.minimum(np.abs(num_grad), np.abs(grad(z))))
    return np.all(rel_tol < rtol), np.max(rel_tol)

In [13]:
z = np.random.normal(size=10000)
check_gradients(sigmoid, sigmoid_prime, z, type='central', rtol=1e-5)

(True, 1.672121682130276e-10)

In [14]:
def step_function(x, margin = 0, label=[0,1]):
    return np.where(x >= margin, label[1], label[0])

### Problem 12 (0.5)
Recap: binary cross-entropy between `target` $y$ and `output` $a$:
$$\displaystyle J(a, y;\theta)=-\sum_{i=1}^m\left(y^{(i)}\log(a_{\theta}^{(i)})+(1-y^{(i)})\log(1-a_{\theta}^{(i)})\right)$$
where `target` $y^{(i)}$ is 0 or 1 and `output` is $a_{\theta}^{(i)}\in [0,1]$
The implementation of the binary cross entropy function used in the lecture notes had a computational drawback.
Describe the problem and fix it using some small value $\varepsilon$.
Fill in the implementation below. 

### Solution:
It is known that $\log(x)$ is not defined if $x = 0$ and $|\log(x)|$ grows rapidly if $x \rightarrow 0$. So, if `output` is close to 0 or to 1, some computational problems might occur. However, we can fix it by adding a small value to `output`. 

In [15]:
def binary_cross_entropy(output, target, eps=1e-15):
    """ Binary cross-entropy loss function
Args:
    output - the predicted model output
    target - targer values
    eps - tolerance parameter 
Returns:
    bce - loss function values
"""
    #### your code (start)
    bse = target*np.log(output + eps) + (1 - target)*np.log(1 - output + eps)
    bse = -bse.mean()
    #### your code (end)
    return bse

### Problem 13 (0.5)
Explain, what happens to the trained weights $\theta$ of the logistic regression, when the classes are linearly separable.
Offer how to fix the logistic regession model in this case. Fill in the implementation of the `logistic_regression`
function.
Do we have to regularize the bias term? Write why not or why yes.

Я не очень поняла вопрос, потому что насколько я понимаю, логистическая реграссия как раз хорошо работает с линейно разделимыми классами. В случае линейно неразделимых классов можно попробовать преобразовать векторы признаков в пространство с большим количеством измерений.\
Баес регулизовать не нужно. Регуляризация служит препятствием переобучению: когда мы подстраиваемся под конкретные данные и получаем огромные коэффициенты при некоторых переменных. Баес сдвигает все точки функции разом, поэтому таких эффектов не будет.

In [24]:
def add_intercept(X_):
    m, _ = X_.shape
    X = np.hstack((np.ones(m)[:,np.newaxis],X_))
    return X

In [25]:
def logistic_regression(X, y, eta=0.001, iter=100, alpha=0.5):
    """Logistic regression
Args:
    X - feature matrix
    y - target vector
    eta - learning rate
    iter - number of iterations
    alpha - regularization parameter
Returns:
    theta - weight estimates
    losses - historical loss data
    thetas - historical thetas data
"""
    _, n = X.shape
    theta = np.zeros(n)
    losses = np.zeros(iter)
    thetas = np.zeros(iter*3).reshape(iter, 3)
    #### your code (start)
    for i in range(iter):
        theta += eta*(y - sigmoid(X@theta)).T@X
        thetas[i] = theta
        losses[i] = binary_cross_entropy(sigmoid(X@theta), y)
    #### your code (end)
    return theta, losses, thetas

In [26]:
def predict(X, theta):
    y_hat = sigmoid(X.dot(theta))
    return step_function(y_hat, 0.5)

In [27]:
# add simulation for toy example
m = 1000
xi1 = norm.rvs(1, 1, size=2 * m).reshape(m, 2)
xi2 = norm.rvs(10, 5, size=2 * m).reshape(m, 2)
y1 = [0] * m
y2 = [1] * m
X = np.vstack((xi1, xi2))
y = np.hstack((y1, y2))

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
dat = {'x1': X_train.T[0], 'x2': X_train.T[1], 'variable': y_train}

In [29]:
dat = pd.DataFrame(dat)
dat['color'] = ['red' if variable == 0 else 'blue' for variable in dat['variable']]

In [30]:
p = ggplot(dat, aes(x='x1', y='x2', group='variable', color='color')) \
        + geom_point() + theme(legend_position='none')
p

In [31]:
X_train = add_intercept(X_train)

In [32]:
theta, losses, thetas = logistic_regression(X_train, y_train, eta=0.001, iter=100, alpha=20.)

In [33]:
X_test = add_intercept(X_test)

### Problem 14 (0.5)
Derive the expression for the decision boundary. Compute `intercept` and `slope` values,
where `x2 = intercept + slope*x1` in terms of `theta`.

In [34]:
#### Write your code here (start)
intercept = -theta[0]/theta[2]
slope = -theta[1]/theta[2]
#### Write your code here (end)

In [35]:
p += geom_abline(intercept=intercept, slope=slope, color='red', linetype='dashed', size=2)
p

In [36]:
thetas = np.array(thetas)

In [37]:
theta_dat = pd.DataFrame(thetas, columns = ['theta0', 'theta1', 'theta2'])
theta_dat['t'] = np.arange(len(theta_dat['theta0']))
theta_names = list(theta_dat.columns[:-1])

In [38]:
def plot_theta(name):
    return ggplot(theta_dat)+ geom_line(aes('t', name))

In [39]:
bunch = GGBunch()
x0, y0 = 0, 0
row = 0
col = 0
width = 350
height = 250
n_cols = 3
for name in theta_names:
    if col == n_cols:
        col = 0
        row += 1
    plot = plot_theta(name)
    bunch.add_plot(plot, x0 + col*width, y0 + row*height, width, height)
    col += 1
bunch.show()

### Problem 15 (1.0)
Let $\theta\in\mathbb{R}$ and $J(\theta)\in\mathbb{R}$. Consider two-sided numerical gradient 
$$\frac{d J(\theta)}{d\theta}\approx\frac{J(\theta+\varepsilon)-J(\theta-\varepsilon)}{2\varepsilon}$$
We can generalize the above formula for the case of $\theta\in\mathbb{R}^{n}$:
$$\frac{\partial}{\partial\theta_1}J(\theta)\approx\frac{J(\theta_1+\varepsilon,\theta_2,\ldots,\theta_n)-J(\theta_1-\varepsilon,\theta_2,\ldots,\theta_n)}{2\varepsilon};\\
\frac{\partial}{\partial\theta_2}J(\theta)\approx\frac{J(\theta_1,\theta_2+\varepsilon,\ldots,\theta_n)-J(\theta_1,\theta_2-\varepsilon,\ldots,\theta_n)}{2\varepsilon};\\
\vdots\\
\frac{\partial}{\partial\theta_n}J(\theta)\approx\frac{J(\theta_1,\theta_2,\ldots,\theta_n+\varepsilon)-J(\theta_1,\theta_2,\ldots,\theta_n-\varepsilon)}{2\varepsilon}$$
Implement the `gradient_checker` function, which takes some function `J` and it's gardient `grad_J` as functions of `theta` and
checks if the relative tolerance `rel_tol` greater than prespecified threshold value `rtol` for each each axis.
This function has to correctly process not only cases when $\theta\in\mathbb{R}^n$, but also the case $\theta\in\mathbb{R}$.

Relative tolerance for two values $x$ and $y$ is $\displaystyle\boxed{\;\frac{|x-y|}{\min\left(|x|, |y|\right)}\;}$

In [67]:
def to_scalar(x, i):
    """Returns i-th element from an array x if x is array, else returns x
Args:
    x - input array
    i - index to return
Returns:
    x[i] if x is an array and x else
"""
    if isinstance(x, (np.ndarray, list)):
        return x[i]
    return x

Use `np.nditer` to iterate over dimensions. If you are going to perform an operation `theta + eps` along some axis,
don't forget to make a copy of `theta`:
```
theta_ = np.array(theta, copy=True)
```

In [68]:
def gradient_checker(J, grad_J, theta, eps=1e-4, rtol=1e-5):
    """Gradient checker for scalar and vector functions
Args:
    J - function of theta
    grad_J - gradient of function J
    theta - the point for which to compute the numerical gradient
    eps - step value in numerical gradient
    rtol - relative tolerance threshold value
Returns:
    error message if the relative tolerance is greater for some axis
    or "Gradient check passed" else
"""
    it = np.nditer(theta, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        ix = it.multi_index

        #num_grad = 0
        #rel_tol = 1
        #### your code (start)
        theta_ = np.array(theta, copy=True)
        theta_[ix] += eps
        num_grad = to_scalar((J(theta_)-J(theta)), ix)/(2*eps)
        gr = to_scalar(grad_J(theta), ix)
        rel_tol = (num_grad-gr)/min(num_grad, gr)
        #### your code (end)


        if rel_tol > rtol:
            print(f'Incorrect gradient for the axis {str(ix)}')
            return
        it.iternext()
    print(f'Gradient check passed')

Check the gradient of sigmoid function.

In [69]:
z = np.random.normal(size=5)
gradient_checker(sigmoid, sigmoid_prime, z, eps=1e-4, rtol=1e-5)

Gradient check passed


Check the gradient of the binary cross entropy function.

Note that both `loss_function` and `loss_grad` are defined as functions of `theta`.

In [70]:
def loss_function(theta, y=y_train, X=X_train):
    y_hat = sigmoid(X.dot(theta))
    return binary_cross_entropy(y_hat, y)

In [71]:
def loss_grad(theta, y=y_train, X=X_train):
    y_hat = sigmoid(X.dot(theta))
    grad = (y - y_hat).dot(X)
    return -grad

In [72]:
m, n = X_train.shape
theta = np.random.normal(scale=0.1, size=n)
gradient_checker(loss_function, loss_grad, theta)

Gradient check passed


# Sigmoid

### Problem 16 (0.5)
Show that 
$$y\log g(\theta^Tx)+(1-y)\log\left(1-g(\theta^Tx)\right)=y\theta^Tx-\log\left(1+\exp(\theta^Tx)\right),$$
where $\displaystyle g(z)=\frac{1}{1+\exp(-z)}$ - sigmoid function.

### Solution:

$$y\log g(\theta^Tx)+(1-y)\log\left(1-g(\theta^Tx)\right)=y \log(\frac{1}{1+\exp(-\theta^Tx)})+ (1-y)\log(\frac{\exp(-\theta^Tx)}{1+\exp(-\theta^Tx)})=y\theta^Tx-\log\left(1+\exp(-\theta^Tx)\right)$$

# Adversarial examples

### Adversarial examples: Fast Gradient Sign Method
See the article [EXPLAINING AND HARNESSING ADVERSARIAL EXAMPLES](https://arxiv.org/pdf/1412.6572.pdf) for the details.
Let the $\theta$ be the parameters of the model with inputs $x$ and target values $y$. The loss function of the model
denote as $J(\theta,x,y)$ then the FGS method of construction of the adversarial examples is:
$$x\leftarrow \mathrm{clip}\left(x+\eta\;\mathrm{sign}\nabla_x J(\theta,x,y)\right)$$
where the function $\mathrm{clip}$ is used to clip each dimension of $x$ to the range $[0,1]$.
Below we have the trained softmax regression.

In [38]:
def sigmoid(x,slope=1.0):
    return 1.0/(1.0+np.exp(-slope*x))

In [39]:
def step_function(x, margin = 0, label=[0,1]):
    return np.where(x >= margin, label[1], label[0])

In [40]:
def mini_batches(X, y, batch_size):
    m, _ = X.shape
    rand_index = np.random.choice(m, size=m)
    X_, y_ = X[rand_index], y[rand_index]
    pos = 0
    while pos < m:
        X_batch, y_batch = X_[pos:pos+batch_size], y_[pos:pos+batch_size]
        yield (X_batch, y_batch)
        pos += batch_size

In [41]:
def softmax(x):
    a = np.amax(x, axis=1)[:, np.newaxis]
    ex = np.exp(x - a)
    ex_sum = np.sum(ex, axis=1)[:, np.newaxis]
    x = ex / ex_sum
    return x

In [42]:
def one_hot_encoder(labels, n_classes=10):
    encoded = np.zeros((labels.size, n_classes))
    encoded[np.arange(labels.size), labels.astype(int)] = 1
    return encoded

In [43]:
def cross_entropy_loss(output, labels, eps=1e-5):
    logs = np.log(output + eps)
    return np.multiply(-labels, logs).sum(axis=1).mean()

In [44]:
X = pd.read_csv('mnist_data.csv').values[:, 1:]
y = pd.read_csv('mnist_target.csv').values[:, 1]

In [45]:
X = X / 255

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
def softmax_regression(X, y, n_classes=10, eta=0.001, iter=100):
    _, n = X.shape
    theta = np.zeros((n, n_classes))
    losses = np.zeros(iter)
    for i in range(iter):
        loss = 0
        for batch in mini_batches(X, y, 1000):
            X_, y_ = batch
            y_ = one_hot_encoder(y_, n_classes)
            y_hat = softmax(X_.dot(theta))
            grad = X_.T.dot(y_ - y_hat)
            theta += grad*eta
            loss += cross_entropy_loss(y_hat, y_)
        losses[i] = loss
    return theta, losses

In [48]:
def predict(X, theta):
    y_hat = softmax(X.dot(theta))
    return np.argmax(y_hat, axis=1), y_hat

In [49]:
theta, losses = softmax_regression(X_train, y_train, eta=0.0001, iter=100)

In [50]:
def softmax(x):
    if len(x.shape) > 1:
        a = np.amax(x, axis=1)[:, np.newaxis]
        ex = np.exp(x - a)
        ex_sum = np.sum(ex, axis=1)[:, np.newaxis]
        x = ex / ex_sum
    else:
        a = np.amax(x)
        ex = np.exp(x - a)
        ex_sum = np.sum(ex)
        x = ex / ex_sum
    return x

In [51]:
def predict(x, theta):
    y_hat = softmax(theta.T.dot(x))
    return np.argmax(y_hat), y_hat

### Problem 17 (1.0)
Implement the fast gradient sign example function below. Explain what you see looking at the confusion matrix after applying the attack. 

In [52]:
def fast_adversarial_example(dig, theta, n=784, n_classes=10, eta=0.01, iters=10):
    y = np.zeros(n_classes)
    y[dig] = 1
    x = np.random.normal(loc=0, scale=0.01, size=n)
    for _ in range(iters):
        pass
        # write your code here (start)

        # write your code here (end)
    return x

In [53]:
example = fast_adversarial_example(8, theta, eta=0.1, iters=5)

In [54]:
def predict(X, theta):
    y_hat = softmax(X.dot(theta))
    return np.argmax(y_hat, axis=1), y_hat

In [55]:
alpha = 0.5

In [56]:
X_spoiled = np.maximum(X_test*(1-alpha), alpha*example)

In [57]:
y_test_model, test_score = predict(X_spoiled, theta)

In [58]:
gg_confusion_matrix(y_test, y_test_model)

In [59]:
x_target = X_spoiled[0]

In [60]:
def predict(x, theta):
    y_hat = softmax(theta.T.dot(x))
    return np.argmax(y_hat), y_hat

In [61]:
y_pred, score = predict(x_target.T, theta)

In [62]:
bunch = GGBunch()
plot = plot_digit(x_target, 'Predicted digit: '+str(y_pred))
bunch.add_plot(plot, 0, 0)
digit_labels = np.arange(10)
dat = pd.DataFrame({'labels':digit_labels, 'score':score})
plot = ggplot(dat) + geom_histogram(aes('labels', 'score'), stat='identity') + scale_x_discrete()
bunch.add_plot(plot, 500, 0)
bunch.show()

# Fisher information

## Problem 18 (1.0)
#### Fisher information matrix
Show, that under the regularity conditions (that we can interchange differentiation and integration) we have, that $$\displaystyle \boxed{I(\theta;\mathrm{X})=\E\left(\frac{\partial}{\partial\theta}\log L(\theta;\mathrm{X})\right)^2=-\E\left(\frac{\partial^2}{\partial\theta^2}\log L(\theta;\mathrm{X})\right)}$$


### Solution:

Уравнение можно переписать в виде:
$$ \int L(\theta;\mathrm{X}) \left(\frac{\partial}{\partial\theta}\log L(\theta;\mathrm{X})\right)^2 dx=-\int L(\theta;\mathrm{X})\left(\frac{\partial^2}{\partial\theta^2}\log L(\theta;\mathrm{X})\right)dx$$
Рассмотрим оба подынтегральных выражения:
$$L(\theta;\mathrm{X})\left(\frac{\partial}{\partial\theta}\log L(\theta;\mathrm{X})\right)^2 = L(\theta;\mathrm{X})\left(\frac{1}{L(\theta;\mathrm{X})}\frac{\partial}{\partial\theta}L(\theta;\mathrm{X})\right)^2 = \frac{1}{L(\theta;\mathrm{X})}\left(\frac{\partial}{\partial\theta}L(\theta;\mathrm{X})\right)^2 $$

$$ L(\theta;\mathrm{X})\left(\frac{\partial^2}{\partial\theta^2}\log L(\theta;\mathrm{X})\right) = L(\theta;\mathrm{X})\left(\frac{\partial}{\partial\theta}\left(\frac{1}{L(\theta;\mathrm{X})}\frac{\partial}{\partial\theta}L(\theta;\mathrm{X})\right)\right) = -\frac{1}{L(\theta;\mathrm{X})} \left(\frac{\partial}{\partial\theta}L(\theta;\mathrm{X})\right)^2 +\frac{\partial^2}{\partial\theta^2}L(\theta;\mathrm{X}) $$

Левая часть выражения сокращается с первым слагаемым правой, осталось доказать, что 
$$\int\frac{\partial^2}{\partial\theta^2}L(\theta;\mathrm{X})dx = 0$$
Это следует из регулярности аналогично тому, что было на паре:
$$\int\frac{\partial^2}{\partial^2\theta}\mathcal{L}(\theta;\mathrm{X})dx\underbrace{=}_{\color{blue}{\textrm{regularity}}}\frac{\partial^2}{\partial^2\theta}\underbrace{\int \mathcal{L}(\theta;\mathrm{X})dx}_{=1}=0$$

# Dataset distillation

### Dataset distillation
Based on the paper [Dataset Distillation](https://arxiv.org/abs/1811.10959).
Consider training set $\{x^{(i)}\}_{i=1}^m$ and some loss function $\ell(x^{(i)},\theta)$, where $\theta$ - model parameters. Find empirical error minimizer:
$$\theta^{*}=\arg\min_{\theta}\frac{1}{m}\sum_{i=1}^m\ell(x^{(i)},\theta)$$

Find distilled training data $\{\tilde{x}^{(i)}\}_{i=1}^{\tilde{m}}$, such that $\tilde{m}\ll m$ with the help of gradient descent:
$$\theta_1=\theta_0-\tilde{\eta}\nabla_{\theta_0}\ell(\tilde{x},\theta_0)$$
define $\displaystyle \mathcal{L}(x,\eta\;|\;\theta_0)=\ell(x,\theta_1)$ and set distilled data to be
$$\tilde{x}^*,\tilde{\eta}^*=\arg\min_{\tilde{x},\tilde{\eta}}\mathcal{L}\left(\tilde{x},\tilde{\eta}\;|\;\theta_0\right)=\arg\min_{\tilde{x},\tilde{\eta}}\ell(x,\theta_1)=\arg\min_{\tilde{x},\tilde{\eta}}\ell\left(x,\theta_0-\tilde{\eta}\nabla_{\theta_0}\ell(\tilde{x},\theta_0)\right)$$
Actually you can perform several SGD steps, updating distilled data $\tilde{x}$ and corresponding learning rate $\tilde{\eta}$:
$$
\tilde{x} \leftarrow \tilde{x} - \alpha\nabla_{\tilde{x}}\mathcal{L}\\
\tilde{\eta} \leftarrow \tilde{\eta} - \alpha\nabla_{\tilde{\eta}}\mathcal{L},
$$
where $\alpha$ - learning rate of your distillation algorithm.


### Algorithm:
**Input:** $\tilde{m}$ - distilled dataset size, $\alpha$ - learning rate, $T$ - number of iterations, $\theta_0$ - initial value for $\theta$ (taken from some distribution $p(\theta_0)$), $\tilde{\eta}_0$ - initial value for $\tilde{\eta}$
1. Initialize $\tilde{x}$ randomly, set $\tilde{\eta} \leftarrow \tilde{\eta}_0$
2. For each step $t$ in $[1, T]$:
3. Sample a batch of real training data $x_t$
4. Compute updated parameters with SGD on distilled data: $\displaystyle \theta_1 = \theta_0-\tilde{\eta}\nabla_{\theta_0}\ell(\tilde{x},\theta_0)$
5. Evaluate the objective function on real training data: $\displaystyle \mathcal{L}=\ell(x_t,\theta_1)$
6. Update distilled data parameters: 
$$
\tilde{x} \leftarrow \tilde{x} - \alpha\nabla_{\tilde{x}}\mathcal{L}\\
\tilde{\eta} \leftarrow \tilde{\eta} - \alpha\nabla_{\tilde{\eta}}\mathcal{L},
$$

### Gradients
* We are dealing with a classification task, so we can take cross-entropy as an objective function 
$$\ell(x,\theta)=-\sum_jy^{(j)}\log(\hat{y}^{(j)}),$$
where $\displaystyle \hat{y}=\mathrm{softmax}(z)$ and $z=\theta^Tx$. We already know, that $\displaystyle \boxed{\;\frac{\partial \ell}{\partial \theta}=X^T(Y-\hat{Y})\;}$

* We also have to take derivatives $\displaystyle \nabla_{x}\mathcal{L}$ and $\displaystyle \nabla_{\eta}\mathcal{L}$
Consider $\mathcal{L}(x,\eta\;|\;\theta_0)=\ell(x,\theta_1)$ and rewrite the cross-entropy loss in the next form:
$$-\sum_jy^{(j)}\log\hat{y}^{(j)}=-\sum_jy^{(j)}\log\left(\mathrm{softmax}\left(\underbrace{\left(\underbrace{\theta_0-\tilde{\eta}\tilde{x}^T(\tilde{y}-\hat{\tilde{y}})}_{=\theta_1}\right)^Tx}_{=z}\right)\right)$$
and we can notice that
$$
\frac{\partial \mathcal{L}}{\partial \tilde{x}}=\frac{\partial\mathcal{L}}{\partial z}\frac{\partial z}{\partial \tilde{x}}\\
\frac{\partial \mathcal{L}}{\partial \tilde{\eta}}=\frac{\partial\mathcal{L}}{\partial z}\frac{\partial z}{\partial \tilde{\eta}}
$$
where, as we already know $\displaystyle \frac{\partial \mathcal{L}}{\partial z}=\hat{y}-y$
Consider
$$
\frac{\partial z}{\partial \tilde{x}}=\frac{\partial}{\partial \tilde{x}}\left(\left(\theta_0-\tilde{\eta}\tilde{x}^T(\tilde{y}-\hat{\tilde{y}})\right)^Tx\right)
$$

### Problem 19 (2.5)

Derive expressions for $\displaystyle \nabla_{\tilde{x}}\mathcal{L}$ and $\displaystyle \nabla_{\tilde{\eta}}\mathcal{L}$ 

### Solution:


### Read MNIST dataset and perform train/test split


In [63]:
X = pd.read_csv('mnist_data.csv').values[:, 1:]
y = pd.read_csv('mnist_target.csv').values[:, 1]

In [64]:
X = X / 255

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Problem 19 (cont)
Fill in the gaps in the implementation of the distillation algorithm below:


In [66]:
def dataset_distill(X, y, n=784, n_classes=10, alpha=0.1, eta=0.01, num_epochs=10):
    xd = np.random.normal(loc=0, scale=0.01, size=(n, n_classes))
    yd = np.eye(n_classes)
    theta = np.random.normal(loc=0, scale=0.01, size=(n, n_classes))

    grad_x = np.zeros_like(xd)
    grad_eta = 0

    for _ in range(num_epochs):
        for batch in mini_batches(X, y, 1000):
            # compute updated parameter on distilled data
            yd_hat = softmax(xd.T.dot(theta))
            grad = xd.dot(yd - yd_hat)
            theta += grad*eta
            # evaluate the objective function on real training data
            X_, y_ = batch
            y_ = one_hot_encoder(y_, n_classes)
            y_hat = softmax(X_.dot(theta))

            # write your code here (start)


            # write your code here (end)
            xd -= alpha * grad_x
            eta -= alpha * grad_eta
    return xd, eta

In [67]:
xd, eta = dataset_distill(X_train, y_train, eta=0.001, num_epochs=100)

In [68]:
images = xd.T
bunch = GGBunch()
size = 200
cols = 5
col = 0
for i, image in enumerate(images):
    col = i // cols
    row = i%cols
    plot = plot_digit(image.reshape(28, 28)) + ggsize(size, size)
    bunch.add_plot(plot, row * size, col * size)
bunch.show()

In [69]:
yd = np.arange(10)

In [70]:
def softmax_regression(X, y, n_classes=10, eta=0.001, num_epochs=100):
    _, n = X.shape
    theta = np.zeros((n, n_classes))
    losses = np.zeros(num_epochs)
    for i in range(num_epochs):
        y_ = one_hot_encoder(y, n_classes)
        y_hat = softmax(X.dot(theta))
        grad = X.T.dot(y_ - y_hat)
        theta += grad*eta
        loss = cross_entropy_loss(y_hat, y_)
        losses[i] = loss
    return theta, losses

In [71]:
theta, losses = softmax_regression(xd.T, yd, eta=eta, num_epochs=400)

In [72]:
num_epochs = len(losses)

In [73]:
losses_data = pd.DataFrame({'epoch': np.arange(num_epochs),'loss':losses})

In [74]:
other_options = xlim(0, num_epochs) + ggsize(500, 300) + ggtitle('Loss')
ggplot(losses_data) + geom_area(aes(x='epoch',y='loss'), alpha=0.2) + other_options

In [75]:
def predict(X, theta):
    y_hat = softmax(X.dot(theta))
    return np.argmax(y_hat, axis=1), y_hat

In [76]:
y_test_model, test_score = predict(X_test, theta)

In [77]:
gg_confusion_matrix(y_test, y_test_model)

In [78]:
def f1_score_micro(conf_matrix):
    num_tags = conf_matrix.shape[0]
    score = 0.
    pr, p, r = 0., 0., 0.
    for tag in range(num_tags):
        pr += conf_matrix[tag, tag]
        p += sum(conf_matrix[tag, :])
        r += sum(conf_matrix[:, tag])
    try:
        score = 2 * pr / (p + r)
    except ZeroDivisionError:
        pass
    return score

In [79]:
cm = confusion_matrix(y_test, y_test_model)
score = f1_score_micro(cm)
print(f'F1 score micro: {score}')

F1 score micro: 0.11851428571428571


### Let's take some random digits for each class

In [80]:
def first(a, value):
    return next(i for i, x in enumerate(a) if x == value)

In [81]:
def find_first(a, values):
    ids = []
    for value in values:
        ids += [first(a, value)]
    return ids

In [82]:
values = np.arange(10)

In [83]:
ids = find_first(y_train, values)

In [84]:
ys = y_train[ids]
xs = X_train[ids]

In [88]:
theta, losses = softmax_regression(xs, ys, num_epochs=400)

In [89]:
num_epochs = len(losses)

In [90]:
losses_data = pd.DataFrame({'epoch': np.arange(num_epochs),'loss':losses})

In [91]:
other_options = xlim(0, num_epochs) + ggsize(500, 300) + ggtitle('Loss')
ggplot(losses_data) + geom_area(aes(x='epoch',y='loss'), alpha=0.2) + other_options

In [92]:
y_test_model, test_score = predict(X_test, theta)

In [93]:
gg_confusion_matrix(y_test, y_test_model)

In [94]:
cm = confusion_matrix(y_test, y_test_model)
score = f1_score_micro(cm)
print(f'F1 score micro: {score}')

F1 score micro: 0.49


# info

### The due date is $\color{blue}{9}$ of March 2022 23:59:59

### Instructions
* To submit the assignment share your workbook to me. Make sure that you provide me "Can edit" access.

* Check, that you found all the tasks in the workbook.  
